In [22]:
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import seaborn as sns
from adjustText import adjust_text # Evite les labels superposés
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, Dropdown
import ipywidgets as widgets
from matplotlib.lines import Line2D
import os
from matplotlib_venn import venn3,venn3_circles
from IPython.display import display, clear_output
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

<IPython.core.display.Javascript object>

In [2]:
path = "./" 

# Transcripto

In [3]:
def volcano_plot(file, pval, l2FC_up, l2FC_down, label_up, label_down):
    
    data = pd.read_csv(path + f"Transcriptomic/{file}", sep='\t')
    data["-log10"] = -(np.log10(data["padj"]))
    data = pd.DataFrame.dropna(data)
    data["diffexpressed"] = "NO"
    # if log2Foldchange > 2 and pvalue < 0.05, set as "UP" 
    data["diffexpressed"][(data["log2FoldChange"] > l2FC_up) & (data["padj"] < pval)] = "UP"
    # if log2Foldchange < -0.6 and pvalue < 0.05, set as "DOWN"
    data["diffexpressed"][(data["log2FoldChange"] < l2FC_down) & (data["padj"] < pval)] = "DOWN"
    data["diffexpressed"][(data["log2FoldChange"] > -1) & (data["log2FoldChange"] < 1)] = "NONE"
    data["label"] = np.nan
    data["label"][data["diffexpressed"] != "NO"] = data["symbol"][data["diffexpressed"] != "NO"]
    mycolors = []
    labels={'UP': 'red','DOWN':'blue','NO':'grey'}
    for i in data['diffexpressed']:
        if i == 'NONE':
            mycolors.append('white')
        elif i == 'UP':
            mycolors.append('red')
        elif i == 'DOWN':
            mycolors.append('blue')
        elif i == "NO":
            mycolors.append('grey')
    
    fig, ax = plt.subplots(figsize=(15,10))

    x = data['log2FoldChange']
    y = data['-log10']
    n = data["symbol"]
    texte = []

    ax.scatter(x, y, c=mycolors)

    for i, txt in enumerate(n):
        if (x.iloc[i] > label_up and y.iloc[i] > label_up) or (x.iloc[i] < label_down and y.iloc[i] > label_up):
            texte += [ax.text(x.iloc[i], y.iloc[i],txt)]

    ax.set_xlabel("log2FoldChange",fontsize=22)
    ax.set_ylabel("-log10",fontsize=22)
    legend_elements = [Line2D([0], [0], marker='o',color='w', label='UP',markerfacecolor='red', markersize=15),
                   Line2D([0], [0], marker='o', color='w', label='Down',markerfacecolor='blue', markersize=15),
                   Line2D([0], [0], marker='o', color='w', label='NO',markerfacecolor='grey', markersize=15)]
    ax.legend(handles=legend_elements, title="Expression", loc="upper right")
    adjust_text(texte)
    return plt.show() 


In [4]:
element = os.listdir(path + "Transcriptomic/")
file = []
for elm in element:
    if elm.endswith(".txt"):
        file.append(elm)
interact(volcano_plot,
         file=file,
         pval = FloatSlider(min=0, max=0.05, step=0.001, value=0.05, description="adj. P-value"),
         l2FC_up = FloatSlider(min=1, max=10, step=0.5, value=2, description="log2 FC UP"),
         l2FC_down = FloatSlider(min=(-10), max=(-1), step=0.5, value=-2, description="log2 FC DOWN"),
         label_up = FloatSlider(min=5, max=10, step=1, value=10, description="label UP"),
         label_down = FloatSlider(min=(-10), max=(-1), step=1, value=-10, description="label DOWN"))


interactive(children=(Dropdown(description='file', options=('all_de_j2_vs_j0_allgenes.txt', 'all_de_j4_vs_j0_a…

<function __main__.volcano_plot(file, pval, l2FC_up, l2FC_down, label_up, label_down)>

In [35]:
def show_table(file):
    if file == 'J2/J0':
        data = pd.read_csv(path + "Transcriptomic/all_de_j2_vs_j0_allgenes.txt", sep='\t')
    elif file == 'J4/J0':
        data = pd.read_csv(path + "Transcriptomic/all_de_j4_vs_j0_allgenes.txt", sep='\t')
    elif file == 'J4/J2':
        data = pd.read_csv(path + "Transcriptomic/all_de_j4_vs_j2_allgenes.txt", sep='\t')
    return data


interact(show_table, file=["J2/J0", "J4/J0", "J4/J2"])

interactive(children=(Dropdown(description='file', options=('J2/J0', 'J4/J0', 'J4/J2'), value='J2/J0'), Output…

<function __main__.show_table(file)>

# Proteo

In [5]:
def volcano_plot_prot(file:str,  pval, l2FC_up, l2FC_down, t_ratio):
    
    abundance_pval_col = f"Abundance Ratio Adj. P-Value: {t_ratio}"
    abundance_ratio_col = f"Abundance Ratio (log2): {t_ratio}"
    data = pd.read_csv(path + f"Proteomique/{file}", sep='\t', decimal=',')
    data["-log10"] = -(np.log10(data[abundance_pval_col]))
    #data = pd.DataFrame.dropna(data)
    data["diffexpressed"] = "NO"
    # if log2Foldchange > 2 and pvalue < 0.05, set as "UP" 
    data["diffexpressed"][(data[abundance_ratio_col] > l2FC_up) & (data[abundance_pval_col] < pval)] = "UP"
    # if log2Foldchange < -0.6 and pvalue < 0.05, set as "DOWN"
    data["diffexpressed"][(data[abundance_ratio_col] < l2FC_down) & (data[abundance_pval_col] < pval)] = "DOWN"
    data["diffexpressed"][(data[abundance_ratio_col] > -1) & (data[abundance_ratio_col] < 1)] = "NONE"
    data["label"] = np.nan
    data["label"][data["diffexpressed"] != "NO"] = data["Gene Symbol"][data["diffexpressed"] != "NO"]
    mycolors = []

    for i in data['diffexpressed']:
        if i == 'NONE':
            mycolors.append('grey')
        elif i == 'UP':
            mycolors.append('red')
        elif i == 'DOWN':
            mycolors.append('blue')
        elif i == "NO":
            mycolors.append('grey')

    fig, ax = plt.subplots(figsize=(15,10))

    x = data[abundance_ratio_col]
    y = data['-log10']
    n = data["Gene Symbol"]
    texte = []

    ax.scatter(x, y, c=mycolors)

    for i, txt in enumerate(n):
        if (x.iloc[i] > 2 and y.iloc[i] > 2) or (x.iloc[i] < -2 and y.iloc[i] > 2):
            texte += [ax.text(x.iloc[i], y.iloc[i],txt)]

    ax.set_xlabel("log2FoldChange",fontsize=22)
    ax.set_ylabel("-log10",fontsize=22)
    legend_elements = [Line2D([0], [0], marker='o',color='w', label='UP',markerfacecolor='red', markersize=15),
                   Line2D([0], [0], marker='o', color='w', label='Down',markerfacecolor='blue', markersize=15),
                   Line2D([0], [0], marker='o', color='w', label='NO',markerfacecolor='grey', markersize=15)]
    ax.legend(handles=legend_elements, title="Expression", loc="lower right")
    adjust_text(texte)
    return plt.show()

In [19]:
element = os.listdir(path + "Proteomique/")

file = []
for elm in element:
    if elm.endswith(".txt"):
        file.append(elm)
        
interact(volcano_plot_prot,
         file= Dropdown(options = file, value=file[0]),
         pval = FloatSlider(min=0, max=0.05, step=0.001, value=0.05, description="adj. P-value"),
         l2FC_up = FloatSlider(min=1, max=3, step=0.5, value=2, description="log2 FC UP"),
         l2FC_down = FloatSlider(min=(-2.5), max=(-1), step=0.5, value=-2, description="log2 FC DOWN"),
         t_ratio=["(T48h) / (T0)", "(T96h) / (T0)", "(T96h) / (T48h)", "(T0) / (T48h)", "(T0) / (T96h)", "(T48h) / (T96h)"])


interactive(children=(Dropdown(description='file', options=('Experience1.txt', 'Experience2.txt'), value='Expe…

<function __main__.volcano_plot_prot(file: str, pval, l2FC_up, l2FC_down, t_ratio)>

In [51]:
pd.read_csv('Proteomique/Experience1.txt', sep='\t', decimal=",").columns

Index(['Protein FDR Confidence: Combined', 'Master', 'Accession',
       'Description', 'Contaminant', 'Sum PEP Score', 'Coverage [%]',
       '# Peptides', '# PSMs', '# Unique Peptides', '# AAs', 'MW [kDa]',
       'calc. pI', 'Score Sequest HT: Sequest HT',
       '# Peptides (by Search Engine): Sequest HT', 'Biological Process',
       'Cellular Component', 'Molecular Function', 'Pfam IDs',
       'Entrez Gene ID', 'Ensembl Gene ID', 'Gene Symbol', 'Chromosome',
       'KEGG Pathways', 'Reactome Pathways', 'WikiPathways',
       '# Protein Pathway Groups', 'Abundance Ratio: (T48h) / (T0)',
       'Abundance Ratio: (T96h) / (T0)', 'Abundance Ratio: (T96h) / (T48h)',
       'Abundance Ratio (log2): (T48h) / (T0)',
       'Abundance Ratio (log2): (T96h) / (T0)',
       'Abundance Ratio (log2): (T96h) / (T48h)',
       'Abundance Ratio P-Value: (T48h) / (T0)',
       'Abundance Ratio P-Value: (T96h) / (T0)',
       'Abundance Ratio P-Value: (T96h) / (T48h)',
       'Abundance Ratio Adj.

In [52]:
def show_table(file):
    if file == 'J2/J0':
        data = pd.read_csv(path + "Proteomique/Experience1.txt", sep='\t')[["Accession","Description","Biological Process","Molecular Function","Ensembl Gene ID","Gene Symbol", "Abundance Ratio (log2): (T48h) / (T0)","Abundance Ratio Adj. P-Value: (T48h) / (T0)"]]
    elif file == 'J4/J0':
        data = pd.read_csv(path + "Proteomique/Experience1.txt", sep='\t')[["Accession","Description","Biological Process","Molecular Function","Ensembl Gene ID","Gene Symbol", "Abundance Ratio (log2): (T96h) / (T0)","Abundance Ratio Adj. P-Value: (T96h) / (T0)"]]
    elif file == 'J4/J2':
        data = pd.read_csv(path + "Proteomique/Experience1.txt", sep='\t')[["Accession","Description","Biological Process","Molecular Function","Ensembl Gene ID","Gene Symbol", "Abundance Ratio (log2): (T96h) / (T48h)","Abundance Ratio Adj. P-Value: (T96h) / (T48h)"]]
    elif file == 'J0/J2':
        data = pd.read_csv(path + "Proteomique/Experience2.txt", sep='\t')[["Accession","Description","Biological Process","Molecular Function","Ensembl Gene ID","Gene Symbol", "Abundance Ratio (log2): (T0) / (T48h)","Abundance Ratio Adj. P-Value: (T0) / (T48h)"]]
    elif file == 'J0/J4':
        data = pd.read_csv(path + "Proteomique/Experience2.txt", sep='\t')[["Accession","Description","Biological Process","Molecular Function","Ensembl Gene ID","Gene Symbol", "Abundance Ratio (log2): (T0) / (T96h)","Abundance Ratio Adj. P-Value: (T0) / (T96h)"]]
    elif file == 'J2/J4':
        data = pd.read_csv(path + "Proteomique/Experience2.txt", sep='\t')[["Accession","Description","Biological Process","Molecular Function","Ensembl Gene ID","Gene Symbol", "Abundance Ratio (log2): (T48h) / (T96h)","Abundance Ratio Adj. P-Value: (T48h) / (T96h)"]]
    return data


interact(show_table, file=["J2/J0", "J4/J0", "J4/J2", "J0/J2", "J0/J4", "J2/J4"])

interactive(children=(Dropdown(description='file', options=('J2/J0', 'J4/J0', 'J4/J2', 'J0/J2', 'J0/J4', 'J2/J…

<function __main__.show_table(file)>

In [8]:
def Venn_plot(file):
    if file == 'Experience1.txt':
        data = pd.read_csv(path+f"Proteomique/{file}", sep='\t', decimal=",")
        D2_D0 = set(data[(data["Abundance Ratio (log2): (T48h) / (T0)"] > 0) & (data["Abundance Ratio Adj. P-Value: (T48h) / (T0)"] < 0.05)]["Gene Symbol"].dropna())
        D4_D0 = set(data[(data["Abundance Ratio (log2): (T96h) / (T0)"] > 0) & (data["Abundance Ratio Adj. P-Value: (T96h) / (T0)"] < 0.05)]["Gene Symbol"].dropna())
        D4_D2 = set(data[(data["Abundance Ratio (log2): (T96h) / (T48h)"] > 0) & (data["Abundance Ratio Adj. P-Value: (T96h) / (T48h)"] < 0.05)]["Gene Symbol"].dropna())
    elif file == 'Experience2.txt':
        data = pd.read_csv(path+f"Proteomique/{file}", sep='\t', decimal=",")
        D2_D0 = set(data[(data["Abundance Ratio (log2): (T0) / (T48h)"] > 0) & (data["Abundance Ratio Adj. P-Value: (T0) / (T48h)"] < 0.05)]["Gene Symbol"].dropna())
        D4_D0 = set(data[(data["Abundance Ratio (log2): (T0) / (T96h)"] > 0) & (data["Abundance Ratio Adj. P-Value: (T0) / (T96h)"] < 0.05)]["Gene Symbol"].dropna())
        D4_D2 = set(data[(data["Abundance Ratio (log2): (T48h) / (T96h)"] > 0) & (data["Abundance Ratio Adj. P-Value: (T48h) / (T96h)"] < 0.05)]["Gene Symbol"].dropna())
    else :
        data1 = pd.read_csv(path + "Transcriptomic/all_de_j2_vs_j0_allgenes.txt", sep='\t')
        data2 = pd.read_csv(path + "Transcriptomic/all_de_j4_vs_j0_allgenes.txt", sep='\t')
        data3 = pd.read_csv(path + "Transcriptomic/all_de_j4_vs_j2_allgenes.txt", sep='\t')
        D2_D0 = set(data1[(data1["log2FoldChange"] > 0) & (data1["padj"] < 0.05)]["symbol"].dropna())
        D4_D0 = set(data2[(data2["log2FoldChange"] > 0) & (data2["padj"] < 0.05)]["symbol"].dropna())
        D4_D2 = set(data3[(data3["log2FoldChange"] > 0) & (data3["padj"] < 0.05)]["symbol"].dropna())
        
    plt.rcParams["figure.figsize"] = [12, 8]
    plt.rcParams["figure.autolayout"] = True
    vd = venn3([D2_D0, D4_D2 , D4_D0],set_labels=("D2 vs D0","D4 vs D2","D4 vs D0"))
    vd.get_patch_by_id("100").set_alpha(0.5)
    vd.get_patch_by_id("001").set_alpha(0.5)
    vd.get_patch_by_id("011").set_alpha(0.5)
    for text in vd.set_labels:
        text.set_fontsize(25)
    for x in range(len(vd.subset_labels)):
        if vd.subset_labels[x] is not None:
            vd.subset_labels[x].set_fontsize(20)
    plt.title("Venn Diagram")
    return plt.show()

interact(Venn_plot, file=["Experience1.txt", "Experience2.txt", "transcripto"])
    
    

interactive(children=(Dropdown(description='file', options=('Experience1.txt', 'Experience2.txt', 'transcripto…

<function __main__.Venn_plot(file)>